In [17]:
# toydata to test simple algorithm
from datasets import Dataset, DatasetDict, concatenate_datasets
from setretrieval.utils.utils import pickload, pickdump
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
from nltk.corpus import wordnet as wn
import random

In [18]:
all_nouns = [word for synset in wn.all_synsets('n') for word in synset.lemma_names()]
all_nouns = [word for word in all_nouns if "_" not in word]

In [19]:
len(all_nouns)

83939

In [30]:
test_nouns = [{'text': word} for word in all_nouns[:1000]]

In [ ]:
nouns100k = Dataset.from_list([{"text": word} for word in all_nouns[1000:101000]])
nouns10k = Dataset.from_list([{"text": word} for word in all_nouns[1000:11000]])
test_nouns.save_to_disk("../propercache/data/datastores/heldoutnouns")
nouns100k.save_to_disk("../propercache/data/datastores/nouns100k")
nouns10k.save_to_disk("../propercache/data/datastores/nouns10k")

Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 1589835.49 examples/s]


In [33]:
# prepare query, pos, negative data for this stuff
def construct_toy_query(wset, query_words=100, datatype="train"):
    querywords = random.choices(wset, k=query_words)
    querywords = [q['text'] for q in querywords]
    query = " ".join(querywords)
    pos = random.choice(querywords)
    while True: 
        neg = random.choice(wset)['text']
        if neg not in querywords:
            break
    if datatype == "train":
        return {'query': query, 'positive': pos, 'negative': neg}
    else:
        return {'question': query, 'pos_chunks': querywords, 'numposchunks': len(querywords)}

In [36]:
allnouns = Dataset.from_list([{"text": word} for word in all_nouns])
allnouns.save_to_disk("../propercache/data/datastores/allnouns")

Saving the dataset (1/1 shards): 100%|██████████| 83939/83939 [00:00<00:00, 3347013.76 examples/s]


In [35]:
numwords = 100
traindata = DatasetDict({
    'train': Dataset.from_list([construct_toy_query(nouns10k, numwords) for _ in tqdm(range(10000))]),
    'test': Dataset.from_list([construct_toy_query(nouns10k, numwords) for _ in tqdm(range(1000))])
})
traindata.save_to_disk(f"../propercache/data/colbert_training/nountraining{numwords}words")

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 61916.77 examples/s]


In [ ]:
eval10 = Dataset.from_list([construct_toy_query(test_nouns, 10, "eval") for _ in tqdm(range(1000))])
eval100 = Dataset.from_list([construct_toy_query(test_nouns, 100, "eval") for _ in tqdm(range(1000))])

eval10.save_to_disk("../propercache/data/evalsets/nountest10/")
eval100.save_to_disk("../propercache/data/evalsets/nountest100/")

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 43075.49 examples/s]


In [ ]:
# TODO need to make eval data

In [26]:
traindata['train'][4]

{'query': 'spoonfeeding plastron spare reforestation crapulence heave running involvement Dasyurus Torpediniformes travelling eco-warfare numismatology onslaught standardisation facelift shower quitclaim Gerrhonotus redbelly IPO Petromyzontidae Ophiophagus crocodile absenteeism angiography schematization aspiration penning secularisation egg follow-through despoliation transference disforestation historiography splurge plunge cryptomonad discipline soar infusion cockateel poll Arizona Fucales seigniory watching viceroyship retroversion nothosaur Actinia free-for-all passing bestowal trial Dicamptodontidae retribution name peckerwood circumvolution hybrid interference nationalisation hang justice break capture composing calf sifting interpenetration olm messiahship whip-snake operation diagramming prickleback Merostomata counting jerking expiration bacillus road regimentation specialization assignment conferva operations orchotomy enjoyment bust metazoan breaking sacrament figuration no